# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#pull weatherPy csv results 
weather_df = pd.read_csv("../WeatherPy/output_results/cities2.csv")
#display
weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Puerto Ayora,3,EC,1588226167,90,-0.74,-90.35,77.00,1.01
1,Rikitea,92,PF,1588226167,85,-23.12,-134.97,79.18,10.92
2,Hobart,75,AU,1588226167,62,-42.88,147.33,57.99,6.93
3,Upernavik,93,GL,1588226167,81,72.79,-56.15,21.88,6.89
4,Longyearbyen,90,SJ,1588226167,95,78.22,15.64,32.00,18.34
...,...,...,...,...,...,...,...,...,...
554,Muborak,12,UZ,1588226251,30,39.26,65.15,77.32,6.96
555,Celestún,56,MX,1588226251,77,20.87,-90.40,80.96,12.39
556,Okha,98,RU,1588226251,81,53.57,142.95,45.05,12.82
557,San Patricio,15,MX,1588226251,78,19.22,-104.70,69.57,5.53


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Conf. gmaps
gmaps.configure(api_key=g_key)

# assign latitude and longitude
LatandLng = weather_df[["Lat", "Lng"]]

# assing humidity to hum
hum = weather_df["Humidity"]

In [4]:
# Heatmap of humidity
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#assin cities that are the same to search
short_city_df = weather_df.loc[(weather_df["Wind Speed"] <= 10) & (weather_df["Cloudiness"] == 0) & \
                                   (weather_df["Max Temp"] >= 70) & (weather_df["Max Temp"] <= 80)].dropna()

short_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
45,Kifrī,0,IQ,1588226172,29,34.70,44.97,77.07,3.15
121,Bolobo,0,CD,1588226185,73,-2.17,16.23,75.97,2.55
269,Erenhot,0,CN,1588226211,17,43.65,111.98,75.85,3.13
291,Pedernales,0,DO,1588226215,63,18.04,-71.74,77.59,3.89
454,Mossamedes,0,AO,1588226237,62,-15.20,12.15,75.94,5.61
496,Jinchang,0,CN,1588226243,7,38.50,102.17,79.99,6.35


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotelinfo_df for hotel names
hotelinfo_df = short_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# include column for hotel names
hotelinfo_df["Hotel Name"] = ""

# Display
hotelinfo_df

,City,Country,Lat,Lng,Hotel Name
45,Kifrī,IQ,34.70,44.97,
121,Bolobo,CD,-2.17,16.23,
269,Erenhot,CN,43.65,111.98,
291,Pedernales,DO,18.04,-71.74,
454,Mossamedes,AO,-15.20,12.15,
496,Jinchang,CN,38.50,102.17,


In [7]:
maps_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [8]:
#go through each row and display
for index, row in hotelinfo_df.iterrows():
    # pull in needed data
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # use the keyword for params
    params["location"] = f"{lat},{lng}"

    # API request for results
    print(f"display the results {index}: {city_name}.")
    response = requests.get(maps_url, params=params).json()
    
    #results
    final_results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Nearby hotel in {city_name} is {final_results[0]['name']}.")
        hotelinfo_df.loc[index, "Hotel Name"] = final_results[0]['name']

    # if there is no hotel available print this
    except (KeyError, IndexError):
        print("Unknown... next.")
        
        print("-------End of Search-------")

display the results 45: Kifrī.
Unknown... next.
-------End of Search-------
display the results 121: Bolobo.
Unknown... next.
-------End of Search-------
display the results 269: Erenhot.
Nearby hotel in Erenhot is Moli International Hotel.
display the results 291: Pedernales.
Nearby hotel in Pedernales is Hotel Vista de las Águilas.
display the results 454: Mossamedes.
Nearby hotel in Mossamedes is Hotel ibis Styles Iu Namibe.
display the results 496: Jinchang.
Nearby hotel in Jinchang is Jinchuan Hotel.


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotelinfo_df.iterrows()]
locations = hotelinfo_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display
fig

Figure(layout=FigureLayout(height='420px'))